## Lab 2c: Multivariate with no independence assumption

In [1]:
import cv2
import scipy.stats as ss
from mpmath import *
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import multivariate_normal
%xmode plain

Exception reporting mode: Plain


In [2]:
def read_img(img_name='train.jpg', mask_name='mask.jpg'):
    
    img = cv2.imread(img_name, 1)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    mask = cv2.imread(mask_name,1)
    mask = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
    
    return np.array(img).astype(float), np.array(mask).astype(float)

In [3]:
img, mask = read_img()

In [4]:
def get_pixels(img, mask, num_pixels):
    
    if num_pixels:
        ncoin = np.sum(mask > 200 )
        ndesk = np.sum(mask <= 200 )
        return ncoin, ndesk
    else:
        coin = img[(mask > 200).nonzero()]
        desk = img[(mask <= 200).nonzero()]
        return coin, desk

In [5]:
def ML(img, mask, percentage, c_or_d):
    
    coin_pxl, desk_pxl = get_pixels(img, mask, False)
    coin_pxl_v = coin_pxl[:,2]
    desk_pxl_v = desk_pxl[:,2]
    coin_pxl_s = coin_pxl[:,1]
    desk_pxl_s = desk_pxl[:,1]
    coin_pxl_h = coin_pxl[:,0]
    desk_pxl_h = desk_pxl[:,0]
    
    
    # proportion of data to access
    num_elements_coin = int(np.floor(percentage*len(coin_pxl)))
    num_elements_desk = int(np.floor(percentage*len(desk_pxl)))
    
    # sigma calculation functions
    calc_mu =  lambda x: np.mean(x)
    calc_sigma = lambda x: np.cov(x)
    
    if  c_or_d:
        # coin paramaters HS
        
        coin_mu_h = calc_mu(coin_pxl_h[0:num_elements_coin])
        coin_mu_s = calc_mu(coin_pxl_s[0:num_elements_coin])
        coin_mu_v = calc_mu(coin_pxl_v[0:num_elements_coin])
        
        coin_pxl_h = coin_pxl_h[:,np.newaxis]
        coin_pxl_s = coin_pxl_s[:,np.newaxis]
        coin_pxl_v = coin_pxl_v[:,np.newaxis]
        
        mus = np.array([coin_mu_h, coin_mu_s, coin_mu_v])
        hsv = np.hstack((np.hstack((coin_pxl_h, coin_pxl_s)), coin_pxl_v))
        cov_matrix = calc_sigma(hsv.T)
        
        print("coin mu is :")
        display(mus)
        print("coin sigma is :")
        display(cov_matrix)
        return mus, cov_matrix
    
    else:
        # desk parameters 
        
        desk_mu_h = calc_mu(desk_pxl_h[0:num_elements_coin])
        desk_mu_s = calc_mu(desk_pxl_s[0:num_elements_coin])
        desk_mu_v = calc_mu(desk_pxl_v[0:num_elements_coin])
        desk_pxl_h = desk_pxl_h[:,np.newaxis]
        desk_pxl_s = desk_pxl_s[:,np.newaxis]
        desk_pxl_v = desk_pxl_v[:,np.newaxis]
        mus = np.array([desk_mu_h, desk_mu_s, desk_mu_v])
        hsv = np.hstack((np.hstack((desk_pxl_h, desk_pxl_s)), desk_pxl_v))
        print(hsv.shape)
        cov_matrix = calc_sigma(hsv.T)
        
        print("desk mu is :")
        display(mus)
        print("desk sigma is :")
        display(cov_matrix)
        return mus, cov_matrix

In [6]:
coin_mu, coin_sigma = ML(img, mask, 0.99, True)
desk_mu, desk_sigma = ML(img, mask, 0.99, False)

coin mu is :


array([ 28.57463203,  20.45532468, 203.10190476])

coin sigma is :


array([[ 263.043415  ,  -85.1797514 , -107.28862717],
       [ -85.1797514 ,  210.43911158, -124.91564469],
       [-107.28862717, -124.91564469, 1155.56934222]])

(1182133, 3)
desk mu is :


array([ 69.87906926,   7.26002165, 121.89974026])

desk sigma is :


array([[1115.07996944,   43.44266321, -102.34701937],
       [  43.44266321,   30.83451645,   13.7546553 ],
       [-102.34701937,   13.7546553 ,  337.81563989]])

In [7]:
coin_distribution = multivariate_normal(mean=coin_mu, cov=coin_sigma)
desk_distribution = multivariate_normal(mean=desk_mu, cov=desk_sigma)

In [8]:
def classify(img, coin_distribution, desk_distribution):
    out_acc = []
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            p_coin = coin_distribution.pdf(img[i, j, :])
            p_desk = desk_distribution.pdf(img[i, j, :])
            nb_prob = p_coin/(p_coin + p_desk)
            if nb_prob > 0.5:
                img[i, j, :] = 255
                out_acc.append(255)
            else:
                img[i, j, :] = 0
                out_acc.append(0)
    
    cv2.imshow('output', img)
    cv2.waitKey()
    cv2.destroyAllWindows()
                
    out_acc = np.array(out_acc)          
    
    return out_acc

In [9]:
mask[ mask >= 200] = 255
mask[mask < 200] = 0
mask = mask.flatten()
out_acc = classify(img, coin_distribution, desk_distribution)
np.sum(mask == out_acc)/len(out_acc)

0.9107560221354166

In [10]:
img1, mask1 = read_img(img_name='val1.jpg', mask_name='val_mask1.png')
mask1[ mask1 >= 200] = 255
mask1[mask1 < 200] = 0
mask1 = mask1.flatten()
out_acc = classify(img1, coin_distribution, desk_distribution)
acc = np.sum(mask1 ==  out_acc)/len(out_acc)
print("validation image 1: ", acc)

###################################
img2, mask2 = read_img(img_name='val2.jpg', mask_name='val_mask2.png')
mask2[ mask2 >= 200] = 255
mask2[mask2 < 200] = 0
mask2 = mask2.flatten()
out_acc = classify(img2, coin_distribution, desk_distribution)
acc = np.sum(mask2 ==  out_acc)/len(out_acc)
print("validation image 1: ", acc)

validation image 1:  0.8177425130208333
validation image 1:  0.9280037434895834
